In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv("rit_summer_2021_pandemic_data_sales.txt",delimiter="|")

In [3]:
dfitem = pd.read_csv("rit_summer_2021_pandemic_data_item.txt",delimiter="|")

In [4]:
dfjoin2 = pd.merge(df,dfitem,on="ITEM_NBR")


In [6]:
dfsort = dfjoin2.sort_values(['DATE_KEY','CATEGORY_NAME']) 

In [7]:
dfnew = dfsort

In [9]:
dfsort.DATE_KEY = pd.to_datetime(dfjoin2.DATE_KEY,format='%Y%m%d')
dfsort

,TRANSACTION_KEY,DATE_KEY,CUSTOMER_ID,ITEM_NBR,UNITS,WEIGHT,SALES,ITEM_DESCRIPTION,PRODUCT_HIERARCHY,DEPARTMENT_NAME,CATEGORY_NAME,CLASS_NAME,IS_ORGANIC,IS_FAMILYPACK,IS_WEGMANSBRAND
4495013,-7643057983252817104,2019-01-06,2334,566509,2,0.0,18.71,"Atkins Protein-Rich Shake, Milk Chocolate Del...",18-77-10,HEALTH & BEAUTY CARE,ACTIVE NUTRITION,READY TO DRINK,0,1,0
4566747,6296626832796142973,2019-01-06,27,616704,2,0.0,13.67,"Orgain Nutritional Shake, Creamy Chocolate Fu...",18-77-10,HEALTH & BEAUTY CARE,ACTIVE NUTRITION,READY TO DRINK,1,0,0
4606776,-6352651405532565133,2019-01-06,1151,432269,1,0.0,3.95,Atkins RTD-Dark Chocolate 44 FO,18-77-10,HEALTH & BEAUTY CARE,ACTIVE NUTRITION,READY TO DRINK,0,0,0
4606801,-8917307825684994968,2019-01-06,5388,432269,1,0.0,3.95,Atkins RTD-Dark Chocolate 44 FO,18-77-10,HEALTH & BEAUTY CARE,ACTIVE NUTRITION,READY TO DRINK,0,0,0
4606811,5828948223572083953,2019-01-06,3103,432269,2,0.0,7.91,Atkins RTD-Dark Chocolate 44 FO,18-77-10,HEALTH & BEAUTY CARE,ACTIVE NUTRITION,READY TO DRINK,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5165518,1631198440307622125,2021-01-02,3483,773070,3,0.0,2.79,"Two Good Yogurt, Greek, Lowfat, Blueberry Fla...",40-10-05,CULTURED DAIRY,YOGURT,GREEK - STRAINED,0,0,0
5198641,-1427671377102453161,2021-01-02,3090,669990,1,0.0,4.31,"So Delicious Coconutmilk, Unsweetened Vanilla...",40-10-50,CULTURED DAIRY,YOGURT,PLANT BASED YOGURTS,1,0,0
5212932,-4140679229051245359,2021-01-02,2464,777686,4,0.0,5.73,"Icelandic Provisions Provision Krimi Skyr, Str...",40-10-05,CULTURED DAIRY,YOGURT,GREEK - STRAINED,0,0,0
5295903,-587184425923067525,2021-01-02,3411,47895,1,0.0,0.71,Wegmans Organic Plain Greek Yogurt Made with W...,40-10-05,CULTURED DAIRY,YOGURT,GREEK - STRAINED,1,0,1


##THE CRUX MOVE! Group and Sum the categories by day

In [10]:
dfnew = dfsort.groupby([pd.Grouper(key='DATE_KEY',freq='d'),'CATEGORY_NAME'])['SALES'].sum().reset_index()

##Feature Engineering

In [11]:
dfnew['Yesterday Sales'] = dfnew.groupby(['CATEGORY_NAME'])['SALES'].shift()
dfnew['Week Ago Sales'] = dfnew.groupby(['CATEGORY_NAME'])['SALES'].shift(7)

In [12]:
dfnew['isWeekend'] = (dfnew['DATE_KEY'].dt.dayofweek >= 5)

In [13]:
dfnew

,DATE_KEY,CATEGORY_NAME,SALES,Yesterday Sales,Week Ago Sales,isWeekend
0,2019-01-06,ACTIVE NUTRITION,139.54,NaN,NaN,True
1,2019-01-06,AIR CARE,42.34,NaN,NaN,True
2,2019-01-06,ALLERGY,98.21,NaN,NaN,True
3,2019-01-06,ANALGESICS,32.17,NaN,NaN,True
4,2019-01-06,ANTI-PERSPIRANT & DEODORANT,73.64,NaN,NaN,True
...,...,...,...,...,...,...
294911,2021-01-02,WHOLE FISH,13.26,21.89,7.62,True
294912,2021-01-02,WILD FISH W/O SALMON,3.33,20.20,98.54,True
294913,2021-01-02,WONKA/WRIGLEY/NOVELTY,1.57,1.25,1.71,True
294914,2021-01-02,YEAST,7.02,7.18,4.57,True


In [17]:
from sklearn.metrics import mean_squared_error
def rmsle(ytrue, ypred):
    return np.sqrt(mean_squared_error(ytrue, ypred))

In [167]:
dfnew.isna().sum()

DATE_KEY              0
CATEGORY_NAME         0
SALES                 0
Yesterday Sales     591
Week Ago Sales     4054
isWeekend             0
dtype: int64

In [21]:
dfnew[(dfnew['DATE_KEY'] == '10-10-19') & dfnew['Yesterday Sales'].isna()] #NA values do exist in there

,DATE_KEY,CATEGORY_NAME,SALES,Yesterday Sales,Week Ago Sales,isWeekend
113622,2019-10-10,CORPORATE COUPON/SPECIAL PROMO,-0.72,NaN,NaN,False


In [22]:
dfnew.groupby('CATEGORY_NAME')['SALES'].sum()

CATEGORY_NAME
ACCESSORIES                  62.38
ACTIVE NUTRITION          44016.61
AD MOD DUMMY ITEMS         -399.60
AIR CARE                  18013.43
ALLERGY                   36796.81
                           ...    
WOMEN'S HEALTH             5488.59
WONKA/WRIGLEY/NOVELTY       731.29
YEAST                      2718.64
YOGURT                   246158.12
ZUZU                        122.23
Name: SALES, Length: 591, dtype: float64

In [23]:
dfnew = dfnew.dropna() # I hesitate to do this, since it seems to uniformly drop the estimates, might be worth playing with fillna()

In [24]:
dfnew.groupby('CATEGORY_NAME')['SALES'].sum() #this has the effect of lowering all sales by a nontrivial amount

CATEGORY_NAME
ACCESSORIES                  45.18
ACTIVE NUTRITION          43556.94
AD MOD DUMMY ITEMS         -322.56
AIR CARE                  17798.69
ALLERGY                   36468.47
                           ...    
WOMEN'S HEALTH             5391.36
WONKA/WRIGLEY/NOVELTY       719.48
YEAST                      2684.53
YOGURT                   243560.71
ZUZU                         53.21
Name: SALES, Length: 568, dtype: float64

## The below is just a baseline, comparing prior day with current

In [42]:
import datetime
start_date = datetime.date(2019,1,14)
end_date = datetime.date(2019,10,25)
delta = datetime.timedelta(days=1)
windowdelta = datetime.timedelta(days=30)
dateidx = start_date

mean_error = []

while dateidx < end_date:
    #print(dateidx)
    train = dfnew[(dfnew['DATE_KEY'] < pd.Timestamp(dateidx)) & (dfnew['DATE_KEY'] > pd.Timestamp(dateidx - windowdelta))]
    val = dfnew[dfnew['DATE_KEY'] == pd.Timestamp(dateidx)]
    
    
    #xtr, xts = train.drop(['SALES'], axis=1), val.drop(['SALES'], axis=1)
    #ytr, yts = train['SALES'].values, val['SALES'].values
    
    p = val['Yesterday Sales'].values

    error = rmsle(val['SALES'].values, p)
 
    print('Day %s - Error %.5f' % (str(dateidx), error))
    mean_error.append(error)
    dateidx += delta
print('Mean Error = %.5f' % np.mean(mean_error))

Day 2019-01-14 - Error 86.22285
Day 2019-01-15 - Error 37.09423
Day 2019-01-16 - Error 202.94639
Day 2019-01-17 - Error 147.07243
Day 2019-01-18 - Error 78.16132
Day 2019-01-19 - Error 109.37478
Day 2019-01-20 - Error 118.60778
Day 2019-01-21 - Error 58.40354
Day 2019-01-22 - Error 104.46336
Day 2019-01-23 - Error 117.09204
Day 2019-01-24 - Error 35.40558
Day 2019-01-25 - Error 41.49322
Day 2019-01-26 - Error 96.52246
Day 2019-01-27 - Error 61.39681
Day 2019-01-28 - Error 70.14565
Day 2019-01-29 - Error 33.90442
Day 2019-01-30 - Error 51.68156
Day 2019-01-31 - Error 49.12879
Day 2019-02-01 - Error 86.60333
Day 2019-02-02 - Error 84.40290
Day 2019-02-03 - Error 123.37755
Day 2019-02-04 - Error 109.14378
Day 2019-02-05 - Error 54.41796
Day 2019-02-06 - Error 35.39649
Day 2019-02-07 - Error 28.00101
Day 2019-02-08 - Error 35.14135
Day 2019-02-09 - Error 39.75818
Day 2019-02-10 - Error 85.19375
Day 2019-02-11 - Error 69.01521
Day 2019-02-12 - Error 96.04986
Day 2019-02-13 - Error 70.32804


Day 2019-10-02 - Error 28.24242
Day 2019-10-03 - Error 27.66407
Day 2019-10-04 - Error 29.41448
Day 2019-10-05 - Error 42.73746
Day 2019-10-06 - Error 49.89955
Day 2019-10-07 - Error 62.39351
Day 2019-10-08 - Error 37.48063
Day 2019-10-09 - Error 25.81900
Day 2019-10-10 - Error 23.79923
Day 2019-10-11 - Error 34.85237
Day 2019-10-12 - Error 44.56337
Day 2019-10-13 - Error 31.15049
Day 2019-10-14 - Error 73.28147
Day 2019-10-15 - Error 49.35356
Day 2019-10-16 - Error 93.38721
Day 2019-10-17 - Error 30.39477
Day 2019-10-18 - Error 65.66707
Day 2019-10-19 - Error 66.84800
Day 2019-10-20 - Error 48.74311
Day 2019-10-21 - Error 79.15987
Day 2019-10-22 - Error 25.60188
Day 2019-10-23 - Error 44.80112
Day 2019-10-24 - Error 34.76975
Mean Error = 66.13973


In [26]:
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

In [29]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(dfnew['CATEGORY_NAME'])
dfnew['CATCODES'] = le.transform(dfnew['CATEGORY_NAME'])

<ipython-input-29-90973c3ab2c7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfnew['CATCODES'] = le.transform(dfnew['CATEGORY_NAME'])


In [30]:
dfnew

,DATE_KEY,CATEGORY_NAME,SALES,Yesterday Sales,Week Ago Sales,isWeekend,CATCODES
2825,2019-01-13,ACTIVE NUTRITION,60.60,103.10,139.54,True,1
2826,2019-01-13,AIR CARE,51.73,41.57,42.34,True,3
2827,2019-01-13,ALLERGY,90.87,55.88,98.21,True,4
2829,2019-01-13,ANALGESICS,57.67,45.41,32.17,True,7
2830,2019-01-13,ANTI-PERSPIRANT & DEODORANT,86.27,56.27,73.64,True,8
...,...,...,...,...,...,...,...
294911,2021-01-02,WHOLE FISH,13.26,21.89,7.62,True,559
294912,2021-01-02,WILD FISH W/O SALMON,3.33,20.20,98.54,True,560
294913,2021-01-02,WONKA/WRIGLEY/NOVELTY,1.57,1.25,1.71,True,564
294914,2021-01-02,YEAST,7.02,7.18,4.57,True,565


In [31]:
# Build predictions for 2019, then compare with actual for 2020
import datetime
pd.set_option('mode.chained_assignment',None) #Turn off SettingWithCopyWarning

start_date = datetime.date(2019,1,14)
end_date = datetime.date(2020,1,13)
delta = datetime.timedelta(days=1)
windowdelta = datetime.timedelta(days=30)
dateidx = start_date

mean_error = []
dfpred = pd.DataFrame() 

while dateidx <= end_date:
    print(dateidx)
    #we will now predict with a 30 day prior window
    train = dfnew[(dfnew['DATE_KEY'] < pd.Timestamp(dateidx)) & (dfnew['DATE_KEY'] > pd.Timestamp(dateidx - windowdelta))]
    val = dfnew[dfnew['DATE_KEY'] == pd.Timestamp(dateidx)] #predictions for 2019
    
    if val.size > 0:
        xtr, xts = train[['CATCODES','Yesterday Sales','Week Ago Sales','isWeekend']], val[['CATCODES','Yesterday Sales','Week Ago Sales','isWeekend']]
        ytr, yts = train['SALES'].values, val['SALES'].values
    
        mdl = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=0)
        mdl.fit(xtr, ytr)

        p = mdl.predict(xts)
    
        #mask = val['DATE_KEY'] == pd.Timestamp(dateidx)
        val.loc[:, 'Prediction'] = np.round(p)
        #print(val)
        dfpred = pd.concat([dfpred,val])
        #print(dfpred.head())
        error = rmsle(yts, p)
        print('Day %s - Error %.5f' % (str(dateidx), error))
        mean_error.append(error)
    dateidx += delta
print('Mean Error = %.5f' % np.mean(mean_error))

2019-01-14
Day 2019-01-14 - Error 44.12887
2019-01-15
Day 2019-01-15 - Error 35.77218
2019-01-16
Day 2019-01-16 - Error 211.03285
2019-01-17
Day 2019-01-17 - Error 94.37598
2019-01-18
Day 2019-01-18 - Error 113.90113
2019-01-19
Day 2019-01-19 - Error 87.39397
2019-01-20
Day 2019-01-20 - Error 133.77838
2019-01-21
Day 2019-01-21 - Error 37.63375
2019-01-22
Day 2019-01-22 - Error 88.55824
2019-01-23
Day 2019-01-23 - Error 179.01090
2019-01-24
Day 2019-01-24 - Error 68.33822
2019-01-25
Day 2019-01-25 - Error 64.24984
2019-01-26
Day 2019-01-26 - Error 127.12842
2019-01-27
Day 2019-01-27 - Error 101.76186
2019-01-28
Day 2019-01-28 - Error 65.19086
2019-01-29
Day 2019-01-29 - Error 31.53931
2019-01-30
Day 2019-01-30 - Error 34.84927
2019-01-31
Day 2019-01-31 - Error 82.55188
2019-02-01
Day 2019-02-01 - Error 45.69409
2019-02-02
Day 2019-02-02 - Error 75.96547
2019-02-03
Day 2019-02-03 - Error 64.66517
2019-02-04
Day 2019-02-04 - Error 94.41910
2019-02-05
Day 2019-02-05 - Error 50.28887
2019-

Day 2019-07-23 - Error 66.63658
2019-07-24
Day 2019-07-24 - Error 27.92943
2019-07-25
Day 2019-07-25 - Error 73.99039
2019-07-26
Day 2019-07-26 - Error 28.32036
2019-07-27
Day 2019-07-27 - Error 68.00957
2019-07-28
Day 2019-07-28 - Error 31.91494
2019-07-29
Day 2019-07-29 - Error 158.80042
2019-07-30
Day 2019-07-30 - Error 32.43038
2019-07-31
Day 2019-07-31 - Error 487.93725
2019-08-01
Day 2019-08-01 - Error 81.04421
2019-08-02
Day 2019-08-02 - Error 46.55724
2019-08-03
Day 2019-08-03 - Error 43.68215
2019-08-04
Day 2019-08-04 - Error 48.46506
2019-08-05
Day 2019-08-05 - Error 132.08924
2019-08-06
Day 2019-08-06 - Error 41.27286
2019-08-07
Day 2019-08-07 - Error 25.30264
2019-08-08
Day 2019-08-08 - Error 264.89161
2019-08-09
Day 2019-08-09 - Error 124.26940
2019-08-10
Day 2019-08-10 - Error 119.99386
2019-08-11
Day 2019-08-11 - Error 53.25505
2019-08-12
Day 2019-08-12 - Error 37.64262
2019-08-13
Day 2019-08-13 - Error 29.69518
2019-08-14
Day 2019-08-14 - Error 26.58371
2019-08-15
Day 2

In [32]:
dfpred.to_csv('2019dailycategorypredictions_30daywindow.csv')

In [33]:
# Migrate 2019 predictions to 2020
import datetime
pd.set_option('mode.chained_assignment',None) #Turn off SettingWithCopyWarning

start_date = datetime.date(2020,1,14)
end_date = datetime.date(2020,12,31)
delta = datetime.timedelta(days=1)
dateidx = start_date

#just predict all days using final xts model from above
while dateidx <= end_date:
    print(dateidx)
    val = dfnew[dfnew['DATE_KEY'] == pd.Timestamp(dateidx)] 
    if val.size > 0:
        xts = val[['CATCODES','Yesterday Sales','Week Ago Sales','isWeekend']]
        p = mdl.predict(xts)
        val.loc[:, 'Prediction'] = np.round(p)
        #print(val)
        dfpred = pd.concat([dfpred,val])
    dateidx += delta

    

2020-01-14
2020-01-15
2020-01-16
2020-01-17
2020-01-18
2020-01-19
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-24
2020-01-25
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-02-29
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12
2020-03-13
2020-03-14
2020-03-15
2020-03-16
2020-03-17
2020-03-18
2020-03-19
2020-03-20
2020-03-21
2020-03-22
2020-03-23
2020-03-24
2020-03-25
2020-03-26
2020-03-27
2020-03-28
2020-03-29
2020-03-30
2020-03-31
2020-04-01
2020-04-02
2020-04-03
2020-04-04
2020-04-05
2020-04-06
2020-04-07
2020-04-08
2020-04-09
2020-04-10
2020-04-11
2020-04-12
2020-04-13

In [34]:
dfpred

,DATE_KEY,CATEGORY_NAME,SALES,Yesterday Sales,Week Ago Sales,isWeekend,CATCODES,Prediction
3257,2019-01-14,ACTIVE NUTRITION,49.26,60.60,120.71,False,1,91.0
3258,2019-01-14,AIR CARE,16.42,51.73,23.67,False,3,41.0
3259,2019-01-14,ALLERGY,60.42,90.87,21.79,False,4,52.0
3260,2019-01-14,ANALGESICS,18.39,57.67,58.68,False,7,71.0
3261,2019-01-14,ANTI-PERSPIRANT & DEODORANT,42.28,86.27,31.15,False,8,56.0
...,...,...,...,...,...,...,...,...
294117,2020-12-31,WILD SALMON,29.48,27.25,26.67,False,561,36.0
294118,2020-12-31,WINE & BAR ACCESSORIES,25.17,9.90,14.38,False,562,9.0
294119,2020-12-31,WOMEN'S HEALTH,7.19,2.87,16.32,False,563,9.0
294120,2020-12-31,YEAST,11.00,5.71,2.28,False,565,8.0


In [35]:
dfpred.to_csv('2019and2020dailycategorypredictions_30daywindow.csv')

In [36]:
mdl.feature_importances_


array([0.03121064, 0.80950722, 0.14469219, 0.01458996])

In [1]:
# Build predictions for 2019, then compare with actual for 2020
import datetime
pd.set_option('mode.chained_assignment',None) #Turn off SettingWithCopyWarning

start_date = datetime.date(2019,1,14)
end_date = datetime.date(2020,1,13)
delta = datetime.timedelta(days=1)
windowdelta = datetime.timedelta(days=30)
dateidx = start_date

mean_error = []
dfpredlgbm = pd.DataFrame() 

while dateidx <= end_date:
    print(dateidx)
    #we will now predict with a 30 day prior window
    train = dfnew[(dfnew['DATE_KEY'] < pd.Timestamp(dateidx)) & (dfnew['DATE_KEY'] > pd.Timestamp(dateidx - windowdelta))]
    val = dfnew[dfnew['DATE_KEY'] == pd.Timestamp(dateidx)] #predictions for 2019
    
    if val.size > 0:
        xtr, xts = train[['CATCODES','Yesterday Sales','Week Ago Sales','isWeekend']], val[['CATCODES','Yesterday Sales','Week Ago Sales','isWeekend']]
        ytr, yts = train['SALES'].values, val['SALES'].values
    
        mdl = LGBMRegressor(n_estimators=1000, learning_rate=0.1)
        mdl.fit(xtr, np.log1p(ytr))

        p = np.expm1(mdl.predict(xts))
    
        #mask = val['DATE_KEY'] == pd.Timestamp(dateidx)
        val.loc[:, 'Prediction'] = np.round(p)
        #print(val)
        dfpredlgbm = pd.concat([dfpred,val])
        #print(dfpred.head())
        error = rmsle(yts, p)
        print('Day %s - Error %.5f' % (str(dateidx), error))
        mean_error.append(error)
    dateidx += delta
print('Mean Error = %.5f' % np.mean(mean_error))

NameError: name 'pd' is not defined

In [38]:
dfpredlgbm

,DATE_KEY,CATEGORY_NAME,SALES,Yesterday Sales,Week Ago Sales,isWeekend,CATCODES,Prediction
3257,2019-01-14,ACTIVE NUTRITION,49.26,60.60,120.71,False,1,91.0
3258,2019-01-14,AIR CARE,16.42,51.73,23.67,False,3,41.0
3259,2019-01-14,ALLERGY,60.42,90.87,21.79,False,4,52.0
3260,2019-01-14,ANALGESICS,18.39,57.67,58.68,False,7,71.0
3261,2019-01-14,ANTI-PERSPIRANT & DEODORANT,42.28,86.27,31.15,False,8,56.0
...,...,...,...,...,...,...,...,...
152815,2020-01-13,WILD FISH W/O SALMON,18.70,34.31,34.94,False,560,35.0
152816,2020-01-13,WILD SALMON,32.00,31.00,4.90,False,561,9.0
152817,2020-01-13,WOMEN'S HEALTH,16.18,7.90,6.10,False,563,7.0
152818,2020-01-13,YEAST,3.42,5.87,9.15,False,565,7.0


In [40]:
mdl.feature_importances_

array([ 9344, 10043,  9487,  1126])

In [41]:
dfpred.to_csv('2019and2020dailycategorypredictions_30daywindow_lbgbm.csv')